In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Sep 22 16:50:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

%matplotlib inline

In [3]:
dtype = 'float32'
tf.keras.backend.set_floatx(dtype)

In [4]:
cifar10 = tf.keras.datasets.cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train.astype(dtype) / 255.0
y_train = y_train.astype(dtype)
X_test = X_test.astype(dtype)  / 255.0
y_test = y_test.astype(dtype)

X_train = np.reshape(X_train, (-1, 3072))
X_test = np.reshape(X_test, (-1, 3072))

X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

170508288/170498071 [==============================] - 4s 0us/step


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_norm = scaler.transform(X)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

X_train_norm = np.reshape(X_train_norm, (-1, 32, 32, 3))
X_test_norm = np.reshape(X_test_norm, (-1, 32, 32, 3))

In [36]:
class Regularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, regularization_penalty, regularization_method):
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method

    def __call__(self, x):
        if self.regularization_method == 'weighted_l1':
            return self.weighted_l1(x)
        elif self.regularization_method == 'group_sparsity':
            return self.group_sparsity(x)
        else:
            raise NotImplementedError(f"Unknown regularization method {self.regularization_method}")
    
    def weighted_l1(self, x):
        # I.e. for a parameter matrix of 4 input and 10 output neurons:
        #
        # [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        #  [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]
        #
        # The scaling vector could be [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        # and the resulting weighted values could be
        #
        # [[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.],
        #  [0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]]
        #
        # Therefore every additional output neuron is regularized more.

        scaling_vector = tf.cumsum(tf.constant(self.regularization_penalty, shape=(x.shape[-1],), dtype=dtype), axis=0)
        weighted_values = scaling_vector * tf.abs(x)
        return tf.reduce_sum(weighted_values)
    
    def group_sparsity(self, x):
        # I.e. for a parameter matrix of 3 input and 5 output neurons:
        #
        # [[1., 1., 1., 1., 1.],
        #  [1., 2., 2., 1., 2.],
        #  [2., 2., 3., 1., 3.]]
        #
        # The resulting vector of group norms is [2., 2., 3., 1., 3.], therefore for
        # every output neuron, its incoming connections form a group.

        group_norms = tf.norm(x, ord=2, axis=0)
        # assert group_norms.shape[0] == x.shape[1]
        return self.regularization_penalty * tf.reduce_sum(group_norms)

    def get_config(self):
        return {'regularization_penalty': float(self.regularization_penalty)}


class CustomLayer(tf.keras.Model):
    def __init__(self, input_shape):
        super().__init__()

        self.inpt_shp = input_shape
    
    def get_input_shape(self):
        if self.inpt_shp is not None:
            return self.inpt_shp
        
        return self.model.get_layer_input_shape(self)


class Dense(CustomLayer):
    def __init__(self, input_units, units, activation, regularization_penalty=0.01, 
                 regularization_method='weighted_l1', kernel_initializer='glorot_uniform', 
                 bias_initializer='zeros', input_shape=None):
        super().__init__(input_shape)

        self.input_units = input_units
        self.units = units
        self.activation = activation
        self.regularization_penalty = regularization_penalty
        self.regularization_method = regularization_method
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        
        self.A = tf.keras.activations.get(activation)
        self.W_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        self.regularizer = Regularizer(self.regularization_penalty, self.regularization_method)
        
        self.W = tf.Variable(
            name='W',
            initial_value=self.W_init(shape=(input_units, units), dtype=dtype),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(units,), dtype=dtype),
            trainable=True)
        
        if self.regularization_method is not None:
            self.add_loss(lambda: self.regularizer(tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)))
    
    def call(self, inputs):
        return self.A(tf.matmul(inputs, self.W) + self.b)
    
    def copy_without_regularization(self):
        copy = Dense(
            self.input_units, 
            self.units, 
            self.activation, 
            regularization_penalty=self.regularization_penalty, 
            regularization_method=None, 
            kernel_initializer=self.kernel_initializer, 
            bias_initializer=self.bias_initializer
        )
        copy.W = self.W
        copy.b = self.b
        return copy
    
    def get_size(self):
        return self.input_units, self.units
    
    def get_param_string():
        param_string = ""
        weights_with_bias = tf.concat([self.W, tf.reshape(self.b, (1, -1))], axis=0)
        max_parameters = tf.math.reduce_max(tf.abs(weights_with_bias), axis=0).numpy()
        magnitudes = np.floor(np.log10(max_parameters))
        for m in magnitudes:
            if m > 0:
                m = 0
            param_string += str(int(-m))
        return param_string


class Conv2D(CustomLayer):
    def __init__(self, input_filters, filters, filter_size, activation, 
                 strides=(1, 1), padding='SAME', kernel_initializer='glorot_uniform',
                 bias_initializer='zeros', input_shape=None):
        super().__init__(input_shape)
    
        self.input_filters = input_filters
        self.filters = filters
        self.filter_size = filter_size
        self.activation = activation
        self.strides = strides
        self.padding = padding
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        
        self.A = tf.keras.activations.get(activation)
        self.filters_init = tf.keras.initializers.get(kernel_initializer)
        self.b_init = tf.keras.initializers.get(bias_initializer)
        
        self.F = tf.Variable(
            name='filters',
            initial_value=self.filters_init(
                shape=(filter_size[0], filter_size[1], input_filters, filters), dtype=dtype
            ),
            trainable=True)
        
        self.b = tf.Variable(
            name='b',
            initial_value=self.b_init(shape=(filters,), dtype=dtype),
            trainable=True)
    
    def call(self, inputs):
        y = tf.nn.conv2d(inputs, self.F, strides=self.strides, padding=self.padding)
        y = tf.nn.bias_add(y, self.b)
        y = self.A(y)
        return y
    
    def copy_without_regularization(self):
        copy = Conv2D(
            self.input_filters,
            self.filters,
            self.filter_size,
            self.activation, 
            strides=self.strides, 
            padding=self.padding, 
            kernel_initializer=self.kernel_initializer, 
            bias_initializer=self.bias_initializer 
        )
        copy.F = self.F
        copy.b = self.b
        return copy
    
    def get_size(self):
        return self.input_filters, self.filters
    
    def get_param_string():
        param_string = ""
        # TODO
        return param_string


class Sequential(tf.keras.Model):
    def __init__(self, layers, activation=None):
        super().__init__()
        
        self.lrs = list()
        for layer in layers:
            self.lrs.append(layer)
            if isinstance(layer, CustomLayer):
                layer.model = self

    def call(self, inputs):
        x = inputs
        for layer in self.lrs:
            x = layer(x)
        return x
    
    def get_layer_input_shape(self, target_layer):
        input = np.random.normal(size=(1,) + self.lrs[0].inpt_shp)
        for layer in self.lrs:
            if layer is target_layer:
                return tuple(input.shape[1:])
            input = layer(input)
        raise Exception("Layer not found in the model.")
    
    def get_layer_sizes(self):
        """
        Returns the sizes of all layers in the model, including the input and output layer.
        """
        layer_sizes = list()
        first_layer = True
        for l in range(len(self.lrs)):
            layer = self.lrs[l]
            if isinstance(layer, CustomLayer):
                layer_size = layer.get_size()
                if first_layer:
                    layer_sizes.append(layer_size[0])
                    first_layer = False
                layer_sizes.append(layer_size[1])
        return layer_sizes
    
    def get_hidden_layer_sizes(self):
        return self.get_layer_sizes()[1:-1]
    
    def remove_regularization(self):
        for l in range(len(self.lrs)):
            layer = self.lrs[l]
            if isinstance(layer, CustomLayer):
                self.lrs[l] = layer.copy_without_regularization()
    
    def get_regularization_penalty(self):
        return self.lrs[-2].regularizer.regularization_penalty
    
    def set_regularization_penalty(self, regularization_penalty):
        for l in range(0, len(self.lrs) - 1):  # Every layer except of the last is regularized
            self.lrs[l].regularizer.regularization_penalty = regularization_penalty
    
    def prune(self, threshold=0.001):
        for l in range(len(self.lrs) - 1):
            layer1 = self.lrs[l]
            layer2 = self.lrs[l + 1]
            
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            weights_with_biases = tf.concat([W1, tf.reshape(b1, (1, -1))], axis=0)
            neurons_are_active = tf.math.reduce_max(tf.abs(weights_with_biases), axis=0) >= threshold
            active_neurons_indices = tf.reshape(tf.where(neurons_are_active), (-1,))
            
            new_W1 = tf.gather(W1, active_neurons_indices, axis=1)
            new_b1 = tf.gather(b1, active_neurons_indices, axis=0)
            new_W2 = tf.gather(W2, active_neurons_indices, axis=0)

            layer1.W = tf.Variable(name='W', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W', initial_value=new_W2, trainable=True)
    
    def grow(self, percentage, min_new_neurons=5, scaling_factor=0.001):   
        for l in range(len(self.lrs) - 1):
            layer1 = self.lrs[l]
            layer2 = self.lrs[l + 1]
       
            W1 = layer1.W.value()
            b1 = layer1.b.value()
            W2 = layer2.W.value()

            n_new_neurons = max(min_new_neurons, int(W1.shape[1] * percentage))

            W1_growth = layer1.W_init(shape=(W1.shape[0], W1.shape[1] + n_new_neurons), dtype=dtype)[:, -n_new_neurons:] * scaling_factor
            b1_growth = layer1.b_init(shape=(n_new_neurons,), dtype=dtype)
            W2_growth = layer2.W_init(shape=(W2.shape[0] + n_new_neurons, W2.shape[1]), dtype=dtype)[-n_new_neurons:, :] * scaling_factor  # TODO is it better to be multiplying here by scaling_factor? It does help with not increasing the max weights of existing neurons when new neurons are added.

            new_W1 = tf.concat([W1, W1_growth], axis=1)
            new_b1 = tf.concat([b1, b1_growth], axis=0)
            new_W2 = tf.concat([W2, W2_growth], axis=0)

            layer1.W = tf.Variable(name='W1', initial_value=new_W1, trainable=True)
            layer1.b = tf.Variable(name='b1', initial_value=new_b1, trainable=True)
            layer2.W = tf.Variable(name='W2', initial_value=new_W2, trainable=True)
    
    def print_neurons(self):
        for layer in self.lrs[:-1]:
            print(layer.get_param_string())
    
    def evaluate(self, x, y, summed_training_loss, summed_training_accuracy, val_dataset):
        # Calculate training loss and accuracy
        if summed_training_loss is not None:
            loss = summed_training_loss / x.shape[0]
        else:
            loss = None
        
        if summed_training_accuracy is not None:
            accuracy = summed_training_accuracy / x.shape[0]
        else:
            accuracy = None
        
        # Calculate val loss and accuracy
        summed_val_loss = 0
        summed_val_accuracy = 0
        n_val_instances = 0
        
        for step, (x_batch, y_batch) in enumerate(val_dataset):
            y_pred = self(x_batch)
            summed_val_loss += tf.reduce_sum(tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred))
            summed_val_accuracy += float(tf.reduce_sum(tf.keras.metrics.sparse_categorical_accuracy(y_batch, y_pred)))
            n_val_instances += x_batch.shape[0]
        
        val_loss = summed_val_loss / n_val_instances
        val_accuracy = summed_val_accuracy / n_val_instances

        return loss, accuracy, val_loss, val_accuracy
    
    def print_epoch_statistics(self, x, y, summed_training_loss, summed_training_accuracy, val_dataset, print_neurons):
        loss, accuracy, val_loss, val_accuracy = self.evaluate(x, y, summed_training_loss, summed_training_accuracy, val_dataset)
        print(f"loss: {loss} - accuracy: {accuracy} - val_loss: {val_loss} - val_accuracy: {val_accuracy} - penalty: {model.get_regularization_penalty()}")
        hidden_layer_sizes = self.get_hidden_layer_sizes()
        print(f"hidden layer sizes: {hidden_layer_sizes}, total units: {sum(hidden_layer_sizes)}")
        if print_neurons:
            self.print_neurons()
    
    def update_history(self, x, y, summed_loss, summed_accuracy, val_dataset, history):
        loss, accuracy, val_loss, val_accuracy = self.evaluate(x, y, summed_loss, summed_accuracy, val_dataset)
        history['loss'].append(loss)
        history['accuracy'].append(accuracy)
        history['val_loss'].append(val_loss)
        history['val_accuracy'].append(val_accuracy)

    def fit(self, x, y, optimizer, epochs, self_scaling_epochs, batch_size, min_new_neurons, validation_data, pruning_threshold=0.001, 
            regularization_penalty_multiplier=1., stall_coefficient=1, growth_percentage=0.2, mini_epochs_per_epoch=1, verbose=True, print_neurons=False):
        train_dataset = tf.data.Dataset.from_tensor_slices((x, y))
        train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

        val_dataset = tf.data.Dataset.from_tensor_slices(validation_data).batch(batch_size)

        history = {
            'loss': list(),
            'accuracy': list(),
            'val_loss': list(),
            'val_accuracy': list(),
        }

        best_val_loss = np.inf
        training_stalled = False
        for epoch in range(epochs):
            summed_loss = 0
            summed_accuracy = 0

            if verbose:
                print("##########################################################")
                print(f"Epoch {epoch + 1}/{epochs}")

            if epoch < self_scaling_epochs:
                if verbose:
                    print("Before growing:")
                    self.print_epoch_statistics(x, y, None, None, val_dataset, print_neurons)

                loss, accuracy, val_loss, val_accuracy = self.evaluate(x, y, val_dataset)
                if val_loss >= best_val_loss * stall_coefficient:
                    if not training_stalled:
                        penalty = self.get_regularization_penalty() * regularization_penalty_multiplier
                        self.set_regularization_penalty(penalty)
                        training_stalled = True
                else:
                    best_val_loss = val_loss
                    training_stalled = False

                self.grow(percentage=growth_percentage, min_new_neurons=min_new_neurons, scaling_factor=pruning_threshold)
                if verbose:
                    print("After growing:")
                    self.print_epoch_statistics(x, y, None, None, val_dataset, print_neurons)
            
            if epoch == self_scaling_epochs:
                self.remove_regularization()

            for mini_epoch in range(mini_epochs_per_epoch):
                for step, (x_batch, y_batch) in enumerate(train_dataset):
                    with tf.GradientTape() as tape:
                        y_pred = self(x_batch, training=True)
                        raw_loss = tf.keras.losses.sparse_categorical_crossentropy(y_batch, y_pred)
                        loss_value = tf.reduce_mean(raw_loss)
                        loss_value += sum(self.losses)  # Add losses registered by model.add_loss

                        summed_loss += tf.reduce_sum(raw_loss)
                        summed_accuracy += float(tf.reduce_sum(tf.keras.metrics.sparse_categorical_accuracy(y_batch, y_pred)))

                    grads = tape.gradient(loss_value, self.trainable_variables)
                    optimizer.apply_gradients(zip(grads, self.trainable_variables))
            
            if epoch < self_scaling_epochs:
                if verbose:
                    print("Before pruning:")
                    self.print_epoch_statistics(x, y, summed_loss, summed_accuracy, val_dataset, print_neurons)
                self.prune(threshold=pruning_threshold)
                if verbose:
                    print("After pruning:")
                    self.print_epoch_statistics(x, y, None, None, val_dataset, print_neurons)
            else:
                if verbose:
                    self.print_epoch_statistics(x, y, summed_loss, summed_accuracy, val_dataset, print_neurons)
            
            self.update_history(x, y, summed_loss, summed_accuracy, val_dataset, history)

        return history

# Convolutions

In [37]:
epochs = 25
self_scaling_epochs = 0
batch_size = 256
min_new_neurons = 20

In [38]:
model = Sequential([
        Conv2D(3, 96, filter_size=(3, 3), activation='selu', 
            strides=(1, 1), padding='SAME', kernel_initializer='lecun_normal', input_shape=X_train_norm[0,:,:,:].shape),
        Conv2D(96, 96, filter_size=(3, 3), activation='selu', 
            strides=(2, 2), padding='SAME', kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.2),
        Conv2D(96, 192, filter_size=(3, 3), activation='selu', 
            strides=(1, 1), padding='SAME', kernel_initializer='lecun_normal'),
        Conv2D(96, 192, filter_size=(3, 3), activation='selu', 
            strides=(2, 2), padding='SAME', kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Flatten(),
        Dense(12288, 256, activation='selu', regularization_penalty=0., 
            regularization_method=None, kernel_initializer='lecun_normal'),
        Dense(256, 10, activation='softmax', regularization_penalty=0., 
            regularization_method=None),
    ])

In [33]:
%%time

model = Sequential([
        Conv2D(3, 96, filter_size=(3, 3), activation='selu', 
            strides=(1, 1), padding='SAME', kernel_initializer='lecun_normal', input_shape=X_train_norm[0,:,:,:].shape),
        Conv2D(96, 96, filter_size=(3, 3), activation='selu', 
            strides=(2, 2), padding='SAME', kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.2),
        Conv2D(96, 192, filter_size=(3, 3), activation='selu', 
            strides=(1, 1), padding='SAME', kernel_initializer='lecun_normal'),
        Conv2D(96, 192, filter_size=(3, 3), activation='selu', 
            strides=(2, 2), padding='SAME', kernel_initializer='lecun_normal'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Flatten(),
        Dense(12288, 256, activation='selu', regularization_penalty=0., 
            regularization_method=None, kernel_initializer='lecun_normal'),
        Dense(256, 10, activation='softmax', regularization_penalty=0., 
            regularization_method=None),
    ])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.fit(X_train_norm, y_train, optimizer, epochs, self_scaling_epochs, batch_size, 
          min_new_neurons, validation_data=(X_test_norm, y_test))

##########################################################
Epoch 1/25
loss: 2.1398861408233643 - accuracy: 0.39144 - val_loss: 1.3741620779037476 - val_accuracy: 0.5007 - penalty: 0.0
hidden layer sizes: [96, 96, 192, 192, 256], total units: 832
##########################################################
Epoch 2/25
loss: 1.3535252809524536 - accuracy: 0.51546 - val_loss: 1.268190860748291 - val_accuracy: 0.5486 - penalty: 0.0
hidden layer sizes: [96, 96, 192, 192, 256], total units: 832
##########################################################
Epoch 3/25
loss: 1.1914558410644531 - accuracy: 0.5725 - val_loss: 1.166611671447754 - val_accuracy: 0.5835 - penalty: 0.0
hidden layer sizes: [96, 96, 192, 192, 256], total units: 832
##########################################################
Epoch 4/25
loss: 1.0351282358169556 - accuracy: 0.63084 - val_loss: 1.087954044342041 - val_accuracy: 0.623 - penalty: 0.0
hidden layer sizes: [96, 96, 192, 192, 256], total units: 832
#####################

In [10]:
# Temto model je na Cifar10 jiz pomerne vyladeny

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(input_shape=X_train_norm[0,:,:,:].shape, filters=96, kernel_size=(3,3), activation='selu', kernel_initializer='lecun_normal'))
model.add(tf.keras.layers.Conv2D(filters=96, kernel_size=(3,3), strides=2, activation='selu', kernel_initializer='lecun_normal'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv2D(filters=192, kernel_size=(3,3), activation='selu', kernel_initializer='lecun_normal'))
model.add(tf.keras.layers.Conv2D(filters=192, kernel_size=(3,3), strides=2, activation='selu', kernel_initializer='lecun_normal'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='selu', kernel_initializer='lecun_normal'))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [11]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

In [12]:
%%time

model.fit(X_train_norm, y_train, epochs=25, batch_size=256, validation_data=(X_test_norm, y_test))

Epoch 1/25
196/196 [==============================] - 9s 46ms/step - loss: 2.0276 - accuracy: 0.3617 - val_loss: 1.4487 - val_accuracy: 0.4768
Epoch 2/25
196/196 [==============================] - 9s 45ms/step - loss: 1.4076 - accuracy: 0.4938 - val_loss: 1.3187 - val_accuracy: 0.5150
Epoch 3/25
196/196 [==============================] - 9s 46ms/step - loss: 1.2646 - accuracy: 0.5483 - val_loss: 1.1942 - val_accuracy: 0.5702
Epoch 4/25
196/196 [==============================] - 9s 46ms/step - loss: 1.1380 - accuracy: 0.5945 - val_loss: 1.1212 - val_accuracy: 0.5984
Epoch 5/25
196/196 [==============================] - 9s 46ms/step - loss: 1.0255 - accuracy: 0.6347 - val_loss: 1.0245 - val_accuracy: 0.6363
Epoch 6/25
196/196 [==============================] - 9s 45ms/step - loss: 0.9390 - accuracy: 0.6672 - val_loss: 0.9999 - val_accuracy: 0.6479
Epoch 7/25
196/196 [==============================] - 9s 44ms/step - loss: 0.8667 - accuracy: 0.6939 - val_loss: 0.9372 - val_accuracy: 0.6731